## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-13-02-28 +0000,bbc,National Trust blames tax rises as it cuts 550...,https://www.bbc.com/news/articles/clym8029nnvo
1,2025-07-11-12-53-16 +0000,bbc,Unite votes to suspend Rayner's membership ove...,https://www.bbc.com/news/articles/cx24de0d9rdo
2,2025-07-11-12-51-21 +0000,bbc,Government code aims to stop unfair parking ch...,https://www.bbc.com/news/articles/c4g2ej63dv1o
3,2025-07-11-12-43-19 +0000,bbc,"Ukraine to receive US air defence systems, say...",https://www.bbc.com/news/articles/crl04200dp4o
4,2025-07-11-12-43-08 +0000,bbc,Gaza's largest functioning hospital facing dis...,https://www.bbc.com/news/articles/cdx5zeywgrgo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,17
270,uk,5
103,brazil,5
363,tariffs,4
408,where,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
78,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,42
50,2025-07-11-02-31-43 +0000,nypost,Trump admin still deciding where it would depo...,https://nypost.com/2025/07/10/us-news/trump-ad...,35
12,2025-07-11-11-21-25 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...,32
30,2025-07-11-08-36-26 +0000,nypost,Legendary California seafood restaurant clashe...,https://nypost.com/2025/07/11/us-news/californ...,32
36,2025-07-11-06-22-59 +0000,nypost,Woman with ‘urgent message’ for President Trum...,https://nypost.com/2025/07/11/us-news/florida-...,31


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
78,42,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
32,28,2025-07-11-08-22-06 +0000,nypost,Georgia teen Sarah Grace Patrick accused of ki...,https://nypost.com/2025/07/11/us-news/georgia-...
6,25,2025-07-11-12-29-49 +0000,nypost,Tom Homan rips violent clash at California pot...,https://nypost.com/2025/07/11/us-news/tom-homa...
34,20,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
23,16,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
12,15,2025-07-11-11-21-25 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...
13,15,2025-07-11-11-21-07 +0000,nypost,Infamous ‘tot mom’ Casey Anthony spotted cozyi...,https://nypost.com/2025/07/11/us-news/casey-an...
47,15,2025-07-11-03-35-21 +0000,nypost,Grieving mom of dead Camp Mystic camper desper...,https://nypost.com/2025/07/10/us-news/grieving...
36,14,2025-07-11-06-22-59 +0000,nypost,Woman with ‘urgent message’ for President Trum...,https://nypost.com/2025/07/11/us-news/florida-...
40,14,2025-07-11-05-24-54 +0000,nypost,A ‘great’ Detroit father’s final wish gets gra...,https://nypost.com/2025/07/11/us-news/detroit-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
